In [16]:
import os
import pandas as pd
import matplotlib.pyplot as plt
from tsfresh import extract_features
from tsfresh.feature_extraction import EfficientFCParameters
from tqdm import tqdm
import numpy as np

# Directorios
PROCESSED_DIR = "../data/processed"
FIGURES_DIR = "../reports/figures"
os.makedirs(FIGURES_DIR, exist_ok=True)

# Función para guardar figura y texto descriptivo
def save_figure_with_description(fig, fig_name, description):
    folder = os.path.join(FIGURES_DIR, fig_name.replace(" ", "_"))
    os.makedirs(folder, exist_ok=True)
    
    image_path = os.path.join(folder, "image.png")
    txt_path = os.path.join(folder, "description.txt")
    
    fig.savefig(image_path)
    plt.close(fig)
    
    with open(txt_path, "w") as f:
        f.write(description)

# Función para cargar todas las curvas de una clase (positive o negative)
def load_curves(label):
    path = os.path.join(PROCESSED_DIR, label)
    files = [f for f in os.listdir(path) if f.endswith(".csv")]
    data = []
    for file in files:
        df = pd.read_csv(os.path.join(path, file))
        df["id"] = file.replace(".csv", "")
        df["label"] = 1 if label == "positive" else 0
        data.append(df)
    return pd.concat(data)

print("Cargando curvas procesadas...")
df_positive = load_curves("positive")
df_negative = load_curves("negative")

df_all = pd.concat([df_positive, df_negative])
print(f"Curvas cargadas: {len(df_all['id'].unique())}")

# ----------------------------------
# Extracción de características con tsfresh
# ----------------------------------

print("Extrayendo características con tsfresh...")
settings = EfficientFCParameters()

# tsfresh espera un DataFrame con columnas: id, time, flux
features = extract_features(df_all[["id", "time", "flux"]], column_id="id", column_sort="time", default_fc_parameters=settings)

# Agregamos la columna label para cada id
labels = df_all.groupby("id")["label"].first()
features["label"] = labels

# Guardamos características en CSV
features.to_csv("../data/processed/features_tsfresh.csv")
print("Características guardadas en data/processed/features_tsfresh.csv")

# ----------------------------------
# Generar gráficos para cada curva
# ----------------------------------

print("Generando gráficos para cada curva...")

for curve_id, group in tqdm(df_all.groupby("id")):
    # Gráfico 1: Curva de luz normalizada
    fig1, ax1 = plt.subplots(figsize=(8,4))
    ax1.plot(group["time"], group["flux"], color="blue")
    ax1.set_title(f"Curva de luz normalizada - {curve_id}")
    ax1.set_xlabel("Tiempo [días]")
    ax1.set_ylabel("Flujo normalizado")
    fig_name1 = f"Curva de luz normalizada - {curve_id}"
    desc1 = ("Este gráfico muestra la curva de luz normalizada de la estrella.\n"
             "El eje X representa el tiempo en días, el eje Y es el flujo normalizado.\n"
             "Se observa la variabilidad típica y posibles tránsitos en las curvas positivas.")
    save_figure_with_description(fig1, fig_name1, desc1)

    # Gráfico 2: Histograma de flujo
    fig2, ax2 = plt.subplots(figsize=(6,4))
    ax2.hist(group["flux"], bins=50, color="green", alpha=0.7)
    ax2.set_title(f"Histograma de flujo - {curve_id}")
    ax2.set_xlabel("Flujo normalizado")
    ax2.set_ylabel("Frecuencia")
    fig_name2 = f"Histograma de flujo - {curve_id}"
    desc2 = ("Histograma que muestra la distribución del flujo normalizado de la curva de luz.\n"
             "Permite visualizar la simetría y dispersión del flujo durante el período observado.")
    save_figure_with_description(fig2, fig_name2, desc2)

print("✅ Gráficos generados y guardados en reports/figures/")



Cargando curvas procesadas...
Curvas cargadas: 32
Extrayendo características con tsfresh...


Feature Extraction: 100%|██████████| 16/16 [00:09<00:00,  1.65it/s]


Características guardadas en data/processed/features_tsfresh.csv
Generando gráficos para cada curva...


100%|██████████| 32/32 [00:20<00:00,  1.58it/s]

✅ Gráficos generados y guardados en reports/figures/
